# Linearinsering av ikke-lineaere ligninger
<div id="ch3:sec4"></div>

Som eksempel, bruker vi problemet ett kjent problem:

<!-- Equation labels as ordinary links -->
<div id="eq:3401"></div>

$$
\begin{equation} \label{eq:3401} \tag{1}
\begin{array}{l}
y''(x) = \frac{3}{2}y^2\\
y(0) = 4 \ , \ y(1) = 1
\end{array}
\end{equation}
$$

der en av losningene er gitt ved:

<!-- Equation labels as ordinary links -->
<div id="eq:3402"></div>

$$
\begin{equation} \label{eq:3402} \tag{2}
y = \frac{4}{(1+x)^2}
\end{equation}
$$

Vi diskretiserer[(1)](#eq:3401) med bruk av sentraldifferanser for $y''(x)$ :

$$
\frac{y_{i+1}-2y_i+y_{i-1}}{h^2}= \frac{3}{2}y_i^2
$$

eller ordnet:

<!-- Equation labels as ordinary links -->
<div id="eq:3403"></div>

$$
\begin{equation} \label{eq:3403} \tag{3}
y_{i-1} - \left( 2+\frac{3}{2}h^2y_i\right)y_i + y_{i+1} = 0
\end{equation}
$$

med h = $\Delta x$

<!-- dom:FIGURE:[fig16.png, width=400, frac=0.6] <div id="fig:316"></div>  -->
<!-- begin figure -->
<div id="fig:316"></div>

<p></p>
<img src="fig16.png" width=400,>

<!-- end figure -->


Vi har delt intervallet $[ 0, 1]$ i $N + 1$ deler der $h=1/(N+1)$ og $x_i = h\cdot i$ , $i = 0,1,...,N+1$. Da $y_0 = 4$ og $y_{N+1}  = 1$, 
blir [(3)](#eq:3403):

<!-- Equation labels as ordinary links -->
<div id="eq:3404"></div>

$$
\begin{equation} \label{eq:3404} \tag{4}
\begin{array}{rl}
-\left( 2+\frac{3}{2}h^2y_1 \right) y_1 + y_2 &= -4 \\
& \dots \\
y_{i-1} - \left(2+\frac{3}{2}h^2y_i\right) y_i+y_{i+1} &= 0 \\
& \dots \\
y_{N-1} - \left( 2 + \frac{3}{2}h^2y_N\right) y_N &= -1 \\
\text{der } i = 2,3,...N-1\\
\end{array}
\end{equation}
$$

In [1]:
import sympy as sp
sp.init_printing()

In [2]:
def Points(i, j):
    expr = 'y' + str(j)
    expr = sp.symbols(expr)
    return expr
    
N = 5
    
yPoints = sp.Matrix(1, N + 2, Points)
yPoints[0] = - 4
yPoints[-1] = - 1
yPoints

In [3]:
h = sp.symbols('h')
def abcDiagonals(i, j):
    y = 'y' + str(i + 1)
    y = sp.symbols(y)
    if i == j:
        # main diagonal
        out = -(2 + sp.Rational(3,2)*h**2*y)
    elif i == j + 1:
        # subdiagonal, directly below main
        out = 1
    elif j == i + 1:
        # superdiagonal, directly above main
        out = 1
    else:
        out = 0
    return out
    
def unknown(i, j):
    expr = 'y' + str(i + 1)
    expr = sp.symbols(expr)
    return expr

In [4]:
A = sp.Matrix(N, N, abcDiagonals)
Y = sp.Matrix(N, 1, unknown)
d = sp.Matrix.zeros(N, 1)
d[0, 0] = -4
d[N - 1, 0] = -1

A, Y, d

Maa altsaa kvitte oss med $y_i$ som forekommer i matrisen. 

## Metoden med etterslep

Metoden med etterslep gaar ut paa aa starte
en iterasjonsprosess, der vi starter med en valgt losning, og bruker forrige verdi av $y_i$ 
som input i matrisen.

In [5]:
h = sp.symbols('h')
def abcDiagonalsLin(i, j):
    y = 'y^p_' + str(i + 1)
    y = sp.symbols(y)
    if i == j:
        # main diagonal
        out = -(2 + sp.Rational(3,2)*h**2*y)
    elif i == j + 1:
        # subdiagonal, directly below main
        out = 1
    elif j == i + 1:
        # superdiagonal, directly above main
        out = 1
    else:
        out = 0
    return out
A_lin = sp.Matrix(N, N, abcDiagonalsLin)
A_lin, Y, d

Hvor $y^p_i$ betyr at vi bruker forrige (previous) verdi av losningen.

### Numerisk losning

La oss for eksempel velge at $y^0_i = 0$, altsaa en vektor med bare 0 verdier.

In [6]:
%matplotlib inline

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy 
import scipy.sparse
import scipy.sparse.linalg

LNWDT=3; FNT=20
plt.rcParams['lines.linewidth'] = LNWDT; plt.rcParams['font.size'] = FNT

x = np.linspace(0, 1, N + 2)
h = x[1]-x[0]
Yp = np.zeros(N)

d = np.zeros(N)
d[0] = -4
d[-1] = -1

superdiagonal = np.ones(N - 1)
subdiagonal = np.ones(N - 1)

y_analytic = 4./(1 + x)**2

for n in range(0):
    maindiagonal = -(2 + 3/2.*Yp*h**2)

    A = scipy.sparse.diags([subdiagonal, maindiagonal, superdiagonal], [-1, 0, 1], format='csc')
    soln = scipy.sparse.linalg.spsolve(A, d)
    Yp = soln
Yp = np.append(4, Yp)
Yp = np.append(Yp, 1)
#sp.Matrix(soln), sp.Matrix(y_analytic)
plt.figure()
plt.plot(x, y_analytic, 'k')
plt.plot(x, Yp, 'r--')
plt.show()

## Newton linearisering

<!-- Equation labels as ordinary links -->
<div id="eq:3427"></div>

$$
\begin{equation} \label{eq:3427} \tag{5}
\text{Setter } y^{m+1}_i = y_i^m + \delta y_i
\end{equation}
$$

der $\delta y_i$ er avviket (residuet) mellom $y_i$ -verdiene for de to iterasjonene.
Ved bruk av [(5)](#eq:3427) :

<!-- Equation labels as ordinary links -->
<div id="eq:3428"></div>

$$
\begin{equation} \label{eq:3428} \tag{6}
\begin{array}{rcl}
\left( y_i^{m+1} \right)^2 &= \left( y_i^m + \delta y_i \right)^2 = (y_i^m)^2 + 2y_i^m \cdot \delta y_i + (\delta y_i^m)^2\\
&\approx (y_i^m)^2 + 2 y_i^m \cdot \delta y_i = y_i^m(2y_i^{m+1}-y_i^m)
\end{array}
\end{equation}
$$

Lineariseringen bestaar i aa neglisjere $(\delta y)^2$ som liten i forhold til de andre leddene.

[(6)](#eq:3428) innsatt i [(4)](#eq:3404) gir folgende system:

<!-- Equation labels as ordinary links -->
<div id="eq:3429"></div>

$$
\begin{equation} \label{eq:3429} \tag{7}
\begin{array}{rcl}
-(2+3y_1^mh^2)y_1^{m+1} + y_2^{m+1} &= -\frac{3}{2}(y_i^mh)^2 -4\\\\
y_{i-1}^{m+1} - (2+3y_i^mh^2)y_i^{m+1} + y_{i+1}^{m+1}&= -\frac{3}{2}(y_i^mh)^2\\\\
y_{N-1}^{m+1}- (2+3y_N^mh^2)y_N^{m+1} &= -\frac{3}{2}(y_N^mh)^2 -1\\
\end{array}
\end{equation}
$$

In [7]:
h = sp.symbols('h')
def abcDiagonalsNewton(i, j):
    y = 'y^p_' + str(i + 1)
    y = sp.symbols(y)
    if i == j:
        # main diagonal
        out = -(2 + 3*h**2*y)
    elif i == j + 1:
        # subdiagonal, directly below main
        out = 1
    elif j == i + 1:
        # superdiagonal, directly above main
        out = 1
    else:
        out = 0
    return out

def d_Newton(i, j):
    y = 'yp_' + str(i + 1) + '^2'
    y = sp.symbols(y)
    expr = -sp.Rational(3,2)*h**2*y
    return expr

A_Newton = sp.Matrix(N, N, abcDiagonalsNewton)
d_Newton = sp.Matrix(N, 1, d_Newton)
d_Newton[0] = d_Newton[0] - 4
d_Newton[N - 1] = d_Newton[N - 1] - 1
A_Newton, Y, d_Newton

### Numerisk losning

Starter paa samme maate som i sted:

In [8]:
x = np.linspace(0, 1, N + 2)
h = x[1]-x[0]

Yp = np.zeros(N)
y_analytic = 4./(1 + x)**2

superdiagonal = np.ones(N-1)
subdiagonal = np.ones(N-1)

for n in range(0):

    maindiagonal = (-3.*Yp*h**2 - 2)
    
    d = -(3./2)*(Yp*h)**2
    d[0] = d[0] - 4
    d[-1] = d[-1] - 1
    
    A = scipy.sparse.diags([subdiagonal, maindiagonal, superdiagonal], [-1, 0, 1], format='csc')
    soln = scipy.sparse.linalg.spsolve(A, d)
    Yp = soln
    

Yp = np.append(4, Yp)
Yp = np.append(Yp, 1)
plt.figure()
plt.plot(x, y_analytic, 'k')
plt.plot(x, Yp, 'r--')
plt.show()

## Repetisjon; skyteteknikk

In [9]:
from ODEschemes import rk4
def dUfunc(Y, t):
    y0 = Y[0]
    y1 = Y[1]
    return np.array([y1, (3./2)*y0**2])

N = 9
x = np.linspace(0, 1, N + 2)

y_analytic = 4./(1 + x)**2

s0 = -3.
Y_0 = [4., s0]

Y_shoot = rk4(dUfunc, Y_0, x)
y_shoot = Y_shoot[:, 0]
phi0 = y_shoot[-1] - 1

s1 = -6.
for n in range(1):
    Y_0 = [4., s1]
    Y_shoot = rk4(dUfunc, Y_0, x)
    y_shoot = Y_shoot[:, 0]
    phi1 = y_shoot[-1] - 1
    print "s1: {0}; phi1: {1}".format(s1, phi1)
    ds = -phi1 *(s1 - s0)/float(phi1 - phi0)
    s0 = s1
    s1 = s1 + ds
    phi0 = phi1
    
plt.figure()
plt.plot(x, y_analytic, 'k')
plt.plot(x, y_shoot, 'r--')
plt.show()

Flere losninger:

In [10]:

sList = np.linspace(-50, 0, 51)
phiList = np.zeros_like(sList)

for n, s in enumerate(sList):
    Y_0 = [4., s]
    Y_shoot = rk4(dUfunc, Y_0, x)
    y_shoot = Y_shoot[:, 0]
    phiList[n] = y_shoot[-1] - 1
    
plt.figure()
plt.plot(sList, phiList)
plt.plot(sList, np.zeros_like(sList), 'k--')
plt.ylim([-10, 5])
plt.show()

In [11]:
s0 = -3.
Y_0 = [4., s0]

Y_shoot = rk4(dUfunc, Y_0, x)
y_shoot = Y_shoot[:, 0]
phi0 = y_shoot[-1] - 1

s1 = -6.
Y_0 = [4., s1]

Y_shoot = rk4(dUfunc, Y_0, x)
y_shoot = Y_shoot[:, 0]
phi1 = y_shoot[-1] - 1

ds = -phi1*(s1 - s0)/float(phi1 - phi0)
s2 = s1 + ds

plt.figure()
plt.plot(sList, phiList)
plt.plot(sList, np.zeros_like(sList), 'k--')
plt.ylim([-10, 30])
plt.plot([s0, s1, s2], [phi0, phi1, 0], 'r-o')
plt.plot([s2], [0], 'go')
plt.show()

## Oppgave

los den lineare differensiallikningen ved bruk av differansemetoder:

<!-- Equation labels as ordinary links -->
<div id="eq:cos"></div>

$$
\begin{equation} \label{eq:cos} \tag{8}
\begin{array}{l}
y''(x) + y(x) = 0\\
y(0) = -1 \ , \ y(\pi) = 1
\end{array}
\end{equation}
$$

ved bruk av sentraldifferanser

$$
\frac{y_{i+1}-2y_i+y_{i-1}}{h^2} + y_i = 0
$$

eller ordnet:

$$
y_{i-1} + \left(h^2 - 2\right)y_i + y_{i+1} = 0
$$

In [12]:
%matplotlib inline

import matplotlib
import matplotlib.pylab as plt

import sympy as sp
import numpy as np
import scipy 
import scipy.sparse
import scipy.sparse.linalg

N = 9
x = np.linspace(0, np.pi, N + 2)
h = x[1] - x[0]

A = scipy.sparse.diags([], [], shape=(N, N), format='csc')
d = np.zeros(N)
d[0] = 
d[-1] = 
yNum = scipy.sparse.linalg.spsolve()

yNum = np.append(-1, yNum)
yNum = np.append(yNum, 1)

plt.figure()
plt.plot(x, -np.cos(x), 'k')
plt.plot(x, yNum, 'r--')
plt.show()

## Oppgave 2

los den ikke-lineare differensiallikningen ved bruk av ettersleps metoden:

$$
\begin{array}{l}
y''(x) = 2y(x)y'(x)\\
y(0) = 0 \ , \ y(\frac{\pi}{4}) = 1
\end{array}
$$

ved bruk av sentraldifferanser paa baade y'' og y':

$$
\frac{y_{i+1}-2y_i+y_{i-1}}{h^2} = 2y_i \left(\frac{y_{i + 1} - y_{i-1}}{2h}\right)
$$

som videre gir:

$$
y_{i+1}-2y_i+y_{i-1} = h y_i \left(y_{i + 1} - y_{i-1}\right)
$$

Los ved aa bruke forrige verdi av $y$ paa hele det ikke-lineare leddet $2y_i \left(\frac{y_{i + 1} - y_{i-1}}{2h}\right)$

In [13]:
%matplotlib inline

import matplotlib
import matplotlib.pylab as plt

import sympy as sp
import numpy as np
import scipy 
import scipy.sparse
import scipy.sparse.linalg

N = 5
x = np.linspace(0, np.pi/4., N + 2)
h = x[1] - x[0]

yp_full = np.linspace(0, 1, N + 2)
yp = yp_full[1:-1]

A = scipy.sparse.diags([], [], shape=(N, N), format='csc')

for n in range(2):
    yp_pluss = yp_full[2:]
    yp_minus = yp_full[0:-2]
    
    d = 
    d[-1] = d[-1] -  
    
    yp = scipy.sparse.linalg.spsolve()
    
    yp_full = np.append(0, yp)
    yp_full = np.append(yp_full, 1)


plt.figure()
plt.plot(x, np.tan(x), 'k')
plt.plot(x, yp_full, 'r--')
plt.show()